## 1. Bibliotecas e módulos úteis

In [1]:
import re
import pandas as pd

from sklearn.utils import shuffle

import spacy

In [2]:
spacy.__version__

'3.0.5'

## 2. Carregando as bases de dados relacionadas ao BC2GM(BioCreative II Gene Mention) _Corpus_:

> [Gene Mention Tagging task](https://genomebiology.biomedcentral.com/articles/10.1186/gb-2008-9-s2-s2) as part of the BioCreative II challenge is concerned with the named entity extraction of gene and gene product mentions in text. The BC2GM corpus contains a total of 24,583 gene entity mentions.

--- 

* Os aquivos estão no formato `.tsv` (_tab-separated values_). Podemos utilizar tanto o método `pd.read_table()` quanto o `pd.read_csv()` para carregá-los;
  Nesse caso, é necessário especificar o separador `sep='\t'`, indicando se tratar de tabulacão; ou o _alias_ para esse parâmetro, `delimiter='\t'`;
* Os arquivos não possuem cabeçalho. Assim, apenas para melhor compreensão dos dados, nomeamos as colunas por "word" e "tag" por meio do parâmetro `names=['word', 'tag']`;
* Ao tentarmos carregar o arquivo de treino, ocorreu o seguinte erro - acreditamos que devido ao seu tamanho:

    ```python
    Error tokenizing data. C error: EOF inside string starting at row 131598
    ```
* A solução encontrada foi utilizar o parâmetro `quoting=3`.

In [3]:
# Carrega a base de treino e validação(dev) juntas
bc2gm_train = pd.read_csv('./datasets/BC2GM/train.tsv', sep='\t', names=['word', 'tag'], quoting=3)

# Carrega a base de teste
bc2gm_valid = pd.read_csv('./datasets/BC2GM/devel.tsv', sep='\t', names=['word', 'tag'])

## 2.1 Análise Exploratória de Dados

In [4]:
# Exibe as primeiras linhas da base de treino
bc2gm_train.head()

,word,tag
0,Immunohistochemical,O
1,staining,O
2,was,O
3,positive,O
4,for,O


In [5]:
# Exibe as linhas iniciais da base de teste
bc2gm_valid.head()

,word,tag
0,Joys,O
1,and,O
2,F,O
3,.,O
4,2,O


In [6]:
# Exibe a distibuição de valores para a coluna "tag" na base de treino
bc2gm_train['tag'].value_counts(dropna=False)

O    318104
I     22104
B     15197
Name: tag, dtype: int64

In [7]:
# Exibe a distibuição de valores para a coluna "tag" na base de treino
bc2gm_valid['tag'].value_counts(dropna=False)

O    63440
I     4437
B     3060
Name: tag, dtype: int64

De acordo com a coluna `tag`, temos uma típica represetação para tarefas de linguística computacional: O formato __IOB__ - um acrônimo para _Inside_, _Outside_, _Beginning_.

[Inside–outside–beginning (tagging)](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging))

In [8]:
print(f'A base de treino possui {bc2gm_train.shape[0]:,} linhas e {bc2gm_train.shape[1]:,} colunas'.replace(',', '.'))
print(f'A base de validação possui {bc2gm_valid.shape[0]:,} linhas e {bc2gm_valid.shape[1]:,} colunas'.replace(',', '.'))

A base de treino possui 355.405 linhas e 2 colunas
A base de validação possui 70.937 linhas e 2 colunas


## 2.2. Procurando por Dados Faltantes

In [9]:
# Verifica se há missing values na base de treino
print('Valores faltantes na base de treino:' , '-' * 37, bc2gm_train.isna().sum(), sep='\n', end='\n\n')

# Verifica se há missing values na base de test
print('Valores faltantes na base de validação:', '-' * 40, bc2gm_valid.isna().sum(), sep='\n')

Valores faltantes na base de treino:
-------------------------------------
word    21
tag      0
dtype: int64

Valores faltantes na base de validação:
----------------------------------------
word    5
tag     0
dtype: int64


In [10]:
# Remove dados faltantes da base de treino e teste
bc2gm_train.dropna(inplace=True)
bc2gm_valid.dropna(inplace=True)

# Sanity check
print(f'A base de treino possui {bc2gm_train.shape[0]:,} linas e {bc2gm_train.shape[1]:,} colunas'.replace(',', '.'))
print(f'A base de teste possui {bc2gm_valid.shape[0]:,} linnas e {bc2gm_valid.shape[1]:,} colunas'.replace(',', '.'))

A base de treino possui 355.384 linas e 2 colunas
A base de teste possui 70.932 linnas e 2 colunas


## 3 Carregando as bases de dados relacionadas ao BC4CHEMD Corpus:

In [11]:
# Carrega a base de treino
bc4chemd_train = pd.read_csv('./datasets/BC4CHEMD/train.tsv', sep='\t', names=['word', 'tag'], quoting=3)

# Carrega a base de validação
bc4chemd_valid = pd.read_csv('./datasets/BC4CHEMD/devel.tsv', sep='\t', names=['word', 'tag'], quoting=3)

In [12]:
# Exibe as primeiras linhas da base de treino
bc4chemd_train.head()

,word,tag
0,DPP6,O
1,as,O
2,a,O
3,candidate,O
4,gene,O


In [13]:
# Exibe as linhas iniciais da base de teste
bc4chemd_valid.head()

,word,tag
0,Models,O
1,to,O
2,predict,O
3,intestinal,O
4,absorption,O


## 3.2. Procurando por Dados Faltantes

In [14]:
# Verifica se há missing values na base de treino
print('Valores faltantes na base de treino:' , '-' * 37, bc4chemd_train.isna().sum(), sep='\n', end='\n\n')

# Verifica se há missing values na base de test
print('Valores faltantes na base de validação:', '-' * 40, bc4chemd_valid.isna().sum(), sep='\n')

Valores faltantes na base de treino:
-------------------------------------
word    45
tag      0
dtype: int64

Valores faltantes na base de validação:
----------------------------------------
word    50
tag      0
dtype: int64


In [15]:
# Remove dados faltantes da base de treino e validação
bc4chemd_train.dropna(inplace=True)
bc4chemd_valid.dropna(inplace=True)

# Sanity check
print(f'A base de treino possui {bc4chemd_train.shape[0]:,} linhas e {bc4chemd_train.shape[1]:,} colunas'.replace(',', '.'))
print(f'A base de validação possui {bc4chemd_valid.shape[0]:,} linhas e {bc4chemd_valid.shape[1]:,} colunas'.replace(',', '.'))

A base de treino possui 893.640 linhas e 2 colunas
A base de validação possui 887.755 linhas e 2 colunas


## 4. Carregando as bases de dados relacionadas ao BC5CDR-disease _Corpus_:

In [16]:
# Carrega a base de treino
bc5cdr_disease_train = pd.read_csv('./datasets/BC5CDR-disease/train.tsv', sep='\t', names=['word', 'tag'], quoting=3)

# Carrega a base de validação
bc5cdr_disease_valid = pd.read_csv('./datasets/BC5CDR-disease/devel.tsv', sep='\t', names=['word', 'tag'])

## 4.1 Análise Exploratória de Dados

In [17]:
bc5cdr_disease_train.head()

,word,tag
0,Selegiline,O
1,-,O
2,induced,O
3,postural,B
4,hypotension,I


In [18]:
bc5cdr_disease_valid.head()

,word,tag
0,22,O
1,-,O
2,oxacalcitriol,O
3,suppresses,O
4,secondary,B


## 4.2. Procurando por Dados Faltantes

In [19]:
# Verifica se há missing values na base de treino
print('Valores faltantes na base de treino:' , '-' * 37, bc5cdr_disease_train.isna().sum(), sep='\n', end='\n\n')

# Verifica se há missing values na base de validação
print('Valores faltantes na base de validação:', '-' * 40, bc5cdr_disease_valid.isna().sum(), sep='\n')

Valores faltantes na base de treino:
-------------------------------------
word    1
tag     0
dtype: int64

Valores faltantes na base de validação:
----------------------------------------
word    0
tag     0
dtype: int64


In [20]:
# Remove dados faltantes da base de treino e validação
bc5cdr_disease_train.dropna(inplace=True)
bc5cdr_disease_valid.dropna(inplace=True)

# Sanity check
print(f'A base de treino possui {bc5cdr_disease_train.shape[0]:,} linhas e {bc5cdr_disease_train.shape[1]:,} colunas'.replace(',', '.'))
print(f'A base de validação possui {bc5cdr_disease_valid.shape[0]:,} linhas e {bc5cdr_disease_valid.shape[1]:,} colunas'.replace(',', '.'))

A base de treino possui 118.169 linhas e 2 colunas
A base de validação possui 117.391 linhas e 2 colunas


## 5. Criação dos arquivos para treinamento do modelo por meio do spaCy
---
Na versão mais recente do spaCy, a transformação de arquivos para o formato `.spacy` é feita via linha de comando.   
É necessario um arquivo `config.cfg` para se utilizar a linha de comando do spaCy.    
Esse arquivo pode ser criado [aqui](https://spacy.io/usage/training#quickstart).    
Daí, basta clicar no ícone de download, salvar o arquivo na pasta do projeto e executar a seguinte linha de comando:
```bash
$ python -m spacy init fill-config base_config.cfg config.cfg
```

#### 4.1. Exibição no _prompt_ após execução da linha de comando acima:
<img src="assets/config.PNG" width=60%>

#### 4.2. As configurações por mim escolhidas: 


<img src="assets/quickstart.png" width=70%>


Se tentarmos gerar os arquivos `.spacy` utilizando somente as _tags_ IOB, será gerado um erro.   
É preciso completar a _tag_ para o __I__ e o __B__.   
Utilizaremos a _tag_ disponível na tabela encaminhada pelo professor Cristian.

In [21]:
bc2gm_train['tag'].replace({'I': 'I-DRUG-PROTEIN', 'B': 'B-DRUG-PROTEIN'}, inplace=True)
bc2gm_valid['tag'].replace({'I': 'I-DRUG-PROTEIN', 'B': 'B-DRUG-PROTEIN'}, inplace=True)

bc4chemd_train['tag'].replace({'I': 'I-CHEMICALS', 'B': 'B-CHEMICALS'}, inplace=True)
bc4chemd_valid['tag'].replace({'I': 'I-CHEMICALS', 'B': 'B-CHEMICALS'}, inplace=True)

bc5cdr_disease_train['tag'].replace({'I': 'I-DISEASE', 'B': 'B-DISEASE'}, inplace=True)
bc5cdr_disease_valid['tag'].replace({'I': 'I-DISEASE', 'B': 'B-DISEASE'}, inplace=True)

In [22]:
print(bc2gm_train['tag'].unique(), bc2gm_valid['tag'].unique(),
      bc4chemd_train['tag'].unique(), bc4chemd_valid['tag'].unique(),
      bc5cdr_disease_train['tag'].unique(), bc5cdr_disease_valid['tag'].unique(), sep='\n')

['O' 'B-DRUG-PROTEIN' 'I-DRUG-PROTEIN']
['O' 'B-DRUG-PROTEIN' 'I-DRUG-PROTEIN']
['O' 'B-CHEMICALS' 'I-CHEMICALS']
['O' 'B-CHEMICALS' 'I-CHEMICALS']
['O' 'B-DISEASE' 'I-DISEASE']
['O' 'B-DISEASE' 'I-DISEASE']


In [23]:
bc2gm_train.shape, bc2gm_valid.shape, bc4chemd_train.shape, bc4chemd_valid.shape, bc5cdr_disease_train.shape, bc5cdr_disease_valid.shape

((355384, 2), (70932, 2), (893640, 2), (887755, 2), (118169, 2), (117391, 2))

In [24]:
# Base de dados reduzida, somente para exemplificação
train = pd.concat([bc2gm_train, bc4chemd_train, bc5cdr_disease_train])
# test = pd.concat([bc2gm_test, bc4chemd_test])
valid = pd.concat([bc2gm_valid, bc4chemd_valid, bc5cdr_disease_valid])


print(f'A base de dados gerada para treino posssui {train.shape[0]:,} linhas e {train.shape[1]:,} colunas'.replace(',', '.'))
print(f'A base de dados gerada para validação possui {valid.shape[0]:,} linhas e {valid.shape[1]:,} colunas'.replace(',', '.'))

A base de dados gerada para treino posssui 1.367.193 linhas e 2 colunas
A base de dados gerada para validação possui 1.076.078 linhas e 2 colunas


In [25]:
# Remove dados que impedem a conversão dos arquivos
pattern = r'\b\t\w*'
mask = train['word'].str.contains(pattern, regex=True)
print('Quantidade de linhas que devem ser removidas na base de treino:', train[mask].shape[0])
train[mask].head()

Quantidade de linhas que devem ser removidas na base de treino: 0


,word,tag


In [26]:
# Remove dados que impedem a conversão dos arquivos
pattern = r'\b\t\w*'
mask = valid['word'].str.contains(pattern, regex=True)
print('Quantidade de linhas que devem ser removidas na base de validação:', valid[mask].shape[0])
valid[mask].head()

Quantidade de linhas que devem ser removidas na base de validação: 51


,word,tag
1490,\tO\nsmall\tO\ndark\tO\ndots\tO\n,O
3888,\tO\ncracked\tO\n-\tO\ntooth\tO\n,O
7121,\tO\nsperm\tO\nfall\tO\n,O
10466,\tO\nportable\tO\nenhancer\tO\ndomain\tO\n,O
13925,\tO\nhistone\tB\ncode\tO\n,O


In [27]:
valid = valid[mask == False].copy()

print(f'A base de dados gerada para treino posssui {train.shape[0]:,} linhas e {train.shape[1]:,} colunas'.replace(',', '.'))
print(f'A base de dados gerada para validação possui {valid.shape[0]:,} linhas e {valid.shape[1]:,} colunas'.replace(',', '.'))

A base de dados gerada para treino posssui 1.367.193 linhas e 2 colunas
A base de dados gerada para validação possui 1.076.027 linhas e 2 colunas


In [28]:
print(train['tag'].unique(), valid['tag'].unique(), sep='\n')

['O' 'B-DRUG-PROTEIN' 'I-DRUG-PROTEIN' 'B-CHEMICALS' 'I-CHEMICALS'
 'B-DISEASE' 'I-DISEASE']
['O' 'B-DRUG-PROTEIN' 'I-DRUG-PROTEIN' 'B-CHEMICALS' 'I-CHEMICALS'
 'B-DISEASE' 'I-DISEASE']


## 5. Salvando os arquivos pré-processados

In [29]:
# Shuffle data
# train = shuffle(train, random_state=0)

# Salva os arquivos pré-processados em formato "iob"
train.to_csv('./datasets/spacy/train.iob', sep='\t', header=0, index=0)
valid.to_csv('./datasets/spacy/valid.iob', sep='\t', header=0, index=0)

In [30]:
pd.read_csv('./datasets/spacy/train.iob', sep='\t', names=['word', 'tag'])['tag'].value_counts(dropna=False)

O                 1257395
I-CHEMICALS         35923
B-CHEMICALS         29478
I-DRUG-PROTEIN      22100
B-DRUG-PROTEIN      15197
B-DISEASE            4182
I-DISEASE            2918
Name: tag, dtype: int64

In [31]:
pd.read_csv('./datasets/spacy/valid.iob', sep='\t', names=['word', 'tag'])['tag'].value_counts(dropna=False)

O                 997216
I-CHEMICALS        34863
B-CHEMICALS        29486
I-DRUG-PROTEIN      4437
B-DISEASE           4243
B-DRUG-PROTEIN      3060
I-DISEASE           2722
Name: tag, dtype: int64

## 6. Convertendo os arquivos `.iob` para o formato `.spacy`
---
Utilizaremos o exemplo disponível no [GitHub do spaCy](https://github.com/explosion/spaCy/tree/master/extra/example_data/ner_example_data)

```bash
$ python -m spacy convert -c iob -s -n 10 -b en_core_web_sm file.iob .
```

**Obs.:** Removemos a opção `-b en_core_web_sm` pois o modelo não estava convergindo com essa opção

Salvaremos os arquivos convertidos em um diretório chamado _datasets/spacy_:
```python
# Converte o arquivo contendo a base de treino
$ python -m spacy convert -c iob -s -n 10 ./datasets/spacy/train.iob ./datasets/spacy

# Converte o arquivo contendo a base de test
$ python -m spacy convert -c iob -s -n 10 ./datasets/spacy/valid.iob ./datasets/spacy

# Em um único comando
python -m spacy convert -c iob -s -n 10 ./datasets/spacy/train.iob ./datasets/spacy && python -m spacy convert -c iob -s -n 10 ./datasets/spacy/valid.iob ./datasets/spacy
```
#### Saída no _prompt_ de comando:

<img src="./assets/iob_para_spacy.png" width=60%>

```python
# Verifica a consistência dos arquivos gerados para treinamento e validação do modelo
$ python -m spacy debug data config.cfg
```
#### Saída no _prompt_ de comando :

<img src="./assets/consistencia_arquivos.png" width=50%>

## 7. Treinamento e Validação do modelo
---
Utilizando o template disponível em [Quickstart](https://spacy.io/usage/training#quickstart)

```bash
$ python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy
```

Para selecionar a GPU usamos a opção `--gpu-id`:
```bash
$ python -m spacy train config.cfg --gpu-id 0
```

Nossa linha de comando para treinamento e validação do modelo:

```bash
$ python -m spacy train config.cfg --gpu-id 0 --output ./datasets/spacy --paths.train ./datasets/spacy/train.spacy --paths.dev ./datasets/spacy/test.spacy
```

#### Saída no _prompt_ de comando:

<img src="./assets/treinamento.png" width=60%>



## 8. Inferência

In [32]:
# Carrega base de dados
corona_dataset = pd.read_csv('./datasets/corona_dataset.csv')

# Remove colunas sem desnecessárias
corona_dataset.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis='columns', inplace=True)

# Remove dados faltantes
corona_dataset.dropna(inplace=True)

# Exibe as linhas inicias da base de dados
corona_dataset.head()

,date,title,category,body,source
0,2020-03-19,Nets' Access to COVID-19 Tests Questioned,Pro Basketball,New York Mayor Bill de Blasio is among those q...,By Reuters
1,March 14,Ireland Announces Second Death From COVID-19,Europe,A second patient has died of the COVID-19 viru...,By Reuters
2,March 6,England to Register COVID-19 as 'Notifiable Di...,Europe,"England will formally register COVID-19, a dis...",By Reuters
3,March 18,"Lakers Reportedly Under Quarantine, Will Test ...",Pro Basketball,Members of the Los Angeles Lakers are under qu...,By Reuters
4,March 17,Factbox: COVID-19 and the New Coronavirus-Fact...,Europe,Social media is awash with myths about how peo...,By Reuters


In [33]:
corpus = ' '.join(corona_dataset['body'])

# Carrega o modelo customizado e cria um objeto nlp
spacy.prefer_gpu()
nlp = spacy.load('./datasets/spacy/model-best')

# Processa o texto
doc = nlp(corpus)

OSError: [E050] Can't find model './datasets/spacy/model-best'. It doesn't seem to be a Python package or a valid path to a data directory.

In [28]:
for token in doc.ents:
    print(f'{token.text:<20} {token.label_:>10}')

COVID-19 virus          PROTEIN
Saturday              CHEMICALS
coronavirus           CHEMICALS
BBC                   CHEMICALS
COVID-19                PROTEIN
COVID-19                PROTEIN
coronavirus           CHEMICALS
President             CHEMICALS
President             CHEMICALS
Saturday              CHEMICALS
’s                    CHEMICALS
”                     CHEMICALS
’s                    CHEMICALS
”                     CHEMICALS
Covid-19                PROTEIN
disaster              CHEMICALS
Sunday                CHEMICALS
”                       PROTEIN
Graduation            CHEMICALS
came                  CHEMICALS
COVID-19                PROTEIN
ibuprofen             CHEMICALS
COVID-19                PROTEIN
lockdown              CHEMICALS
COVID-19                PROTEIN
Alliance              CHEMICALS
coronavirus           CHEMICALS
COVID-19                PROTEIN
ATP                   CHEMICALS
U. A                  CHEMICALS
COVID-19                PROTEIN
U.S.    

In [29]:
# entities_cat_1 = {"GGP":"#F9E79F", "SO":"#F7DC6F", "TAXON":"#F4D03F", "CHEBI":"#FAD7A0", "GO":"#F8C471", "CL":"#F5B041"}
# entities_cat_2 = {"DNA":"#82E0AA", "CELL_TYPE":"#AED6F1", "CELL_LINE":"#E8DAEF", "RNA":"#82E0AA", "PROTEIN":"#82E0AA"}
# entities_cat_3 = {"DISEASE":"#D7BDE2", "CHEMICAL":"#D2B4DE"}
# entities_cat_4 = {"CANCER":"#ABEBC6", "ORGAN":"#82E0AA", "TISSUE":"#A9DFBF", "ORGANISM":"#A2D9CE", "CELL":"#76D7C4", \
#                   "AMINO_ACID":"#85C1E9", "GENE_OR_GENE_PRODUCT":"#AED6F1", "SIMPLE_CHEMICAL":"#76D7C4", "ANATOMICAL_SYSTEM":"#82E0AA", \
#                   "IMMATERIAL_ANATOMICAL_ENTITY":"#A2D9CE", "MULTI-TISSUE_STRUCTURE":"#85C1E9", "DEVELOPING_ANATOMICAL_STRUCTURE":"#A9DFBF", \
#                   "ORGANISM_SUBDIVISION":"#58D68D", "CELLULAR_COMPONENT":"#7FB3D5"}

options = {'ents':['CHEMICALS', 'DRUG_PROTEIN', 'DISEASE', 'SPECIES'],

           'colors':{'CHEMICALS': '#D2B4DE',
                     'DRUG_PROTEIN': '#82E0AA',
                     'DISEASE': '#D7BDE2',
                     'SPECIES': '#A2D9CE'}}

spacy.displacy.render(doc, style="ent", jupyter=True, options=options)

Referências:

[SpaCy](https://spacy.io/)   
[Building a custom NER model in Spacy v3.1](https://zachlim98.github.io/me/2021-03/spacy3-ner-tutorial)   
[NER @ CLI: Custom-named entity recognition with spaCy in four lines](https://blog.codecentric.de/en/2020/11/ner-cli-custom-named-entity-recognition-with-spacy-in-four-lines/)   
[Descubre Named Entity Recognition (NER) con Spacy 3.0 (en castellano)](https://nymiz.com/blog/tecnologia/que-es-el-named-entity-recognition-ner/)




$ python -m spacy debug data config.cfg